In [1]:
from openai import OpenAI
import pandas as pd
import numpy as np
import json
import os
import re
import time
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate #where we change the AI "personality"
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [3]:
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key = os.getenv("api.txt")
COMPLETIONS_MODEL = "text-davinci-002"
BETTER_COMPLETIONS_MODEL = "text-davinci-003" #for my purposes, this is better
LONG_MODEL = "gpt-3.5-turbo-16k"
REGULAR_MODEL = "gpt-3.5-turbo"
GPT_4 = "gpt-4-1106-preview"

In [4]:
os.chdir('/Users/chrissoria/Documents/Research/Categorization_AI_experiments')
current_directory = os.getcwd()
print(current_directory)

/Users/chrissoria/Documents/Research/Categorization_AI_experiments


In [5]:
survey_participant_input = "a19i" #enter column name here

UCNets = pd.read_excel("/Users/chrissoria/Documents/Research/UCNets_Classification/data/Raw_Cond_for_Coding_all_waves.xlsx", engine='openpyxl',sheet_name="JOINT_DATA",usecols=[survey_participant_input])
UCNets = UCNets[survey_participant_input].dropna().unique()  # Drop NaN values and get unique elements

survey_participant_responses = '; '.join(str(item) for item in UCNets) #what we will feed to the model

UCNets = pd.DataFrame(UCNets, columns=[survey_participant_input])
UCNets[survey_participant_input] = UCNets[survey_participant_input].astype(str).str.lower()
UCNets[survey_participant_input] = UCNets[survey_participant_input].str.strip()
UCNets = UCNets[UCNets[survey_participant_input] != ''].reset_index(drop=True) #trimming all empty rows

UCNets = UCNets.iloc[:3]

UCNets.head()

,a19i
0,relocated back to east coast - closer to my sons
1,move in together with my partner
2,"out of living with my friends, and into living..."


Here, I'm trying to "force" the model to "think" in steps by first A. trying to process the response into its own words and B. having it interact with that object. That is, instead of all steps being given at once, I'm having it think in steps. 

This time, I will have it think in a "chain," where I will have it output a response and then feed that response back to it in a seperate prompt. 

In [6]:
survey_question = "Why did you move?"
survey_input = UCNets['a19i']

creativity = .0

user_categories = ["to start or continue living with with a partner/spouse",
                   "related to the person's job, school or career, including transfers, retirement, a new job, or wanting to be closer to work",
                   "related to their partner's job, school or career, such as transfers, retirement, a new job, wanting to be closer to work",
                   "financial reasons, such as increases in rent, affordability of the current housing expenses, pay raises",
                   "related to family members for various reasons, such as providing support, facilitating care, a child's schooling needs, or a new baby"]

user_model = 'gpt-4-1106-preview'

In [13]:
def extract_categories_fs(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(user_categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_response = "the rent was increasing; they wanted to renew the rent $400 extra, a month to re-lease. made sense regarding my career. and i wanted a backyard for my dog."
    example_categorization = """{"1":"0","2":"1","3":"0","4":"1","5":"0"}"""
    example_response2 = "lease ended at my old apartment and i wanted to move back to my parents house to pay off more of my student loans"
    example_categorization2 = """{"1":"0","2":"0","3":"0","4":"1","5":"1"}"""
    example_response3 = "there was a fire in building where i previously lived; all tenants displaced, we had to find other housing. after the fire i stayed 3 days with a friend, then 2 months in a hotel, then began living in my current apartment in same city. move was not by choice, was circumstantial."
    example_categorization3 = """{"1":"0","2":"0","3":"0","4":"0","5":"0"}"""
    
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""Categorize this survey response "{response}" into all of the following reasons for moving and select all that apply: \
        {categories_str} \
        Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values. \
        Here are three examples of a correct categorization. \
        Example survey response 1: {example_response}. \
        Example categorization 1: {example_categorization}. \
        Example survey response 2: {example_response2}. \
        Example categorization 2: {example_categorization2}. \
        Example survey response 3: {example_response3}. \
        Example categorization 3: {example_categorization3}."""
        try:
            response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {'role': 'user', 'content': prompt}
                ],
                temperature=creativity
            )

            reply = response.choices[0].message.content
            link1.append(reply)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {input}")
            
        extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
            
        if extracted_json:
            cleaned_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
            extracted_jsons.append(cleaned_json)
            print(cleaned_json)
        else:
            error_message = """{"1":"e"}"""
            extracted_jsons.append(error_message)
            print(error_message)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            normalized_data_list.append("""{"1":"e"}""")
            continue
            
    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

In [14]:
fewshot = extract_categories_fs(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

fewshot

{"1":"0","2":"0","3":"0","4":"0","5":"1"}
{"1":"1","2":"0","3":"0","4":"0","5":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0"}


,survey_response,link1,json,1,2,3,4,5
0,relocated back to east coast - closer to my sons,"Based on the survey response ""relocated back t...","{""1"":""0"",""2"":""0"",""3"":""0"",""4"":""0"",""5"":""1""}",0,0,0,0,1
1,move in together with my partner,"Based on the survey response ""move in together...","{""1"":""1"",""2"":""0"",""3"":""0"",""4"":""0"",""5"":""0""}",1,0,0,0,0
2,"out of living with my friends, and into living...","Based on the survey response ""out of living wi...","{""1"":""1"",""2"":""0"",""3"":""0"",""4"":""0"",""5"":""0""}",1,0,0,0,0
